In [172]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle
import mlflow
import mlflow.keras

In [173]:
# print("numpy=="+np.__version__)
# print("tensorflow=="+tf.__version__)
# print("pandas=="+pd.__version__)
# print("mlflow=="+mlflow.__version__)


In [174]:
bs=10
epochs=150

In [175]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [176]:



mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("graph-neural-network-trial-batch-norm")

<Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='graph-neural-network-trial-batch-norm', tags={}>

In [177]:
mlflow.tensorflow.autolog()

In [178]:
# def adjm(h,w,near):
#   am=[[0 for i in range(w*h)] for j in range(h*w)]
#   img=[[0 for i in range(w)] for j in range(h)]
  
#   dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

#   for i,v in enumerate(img):
#     for j,v2 in enumerate(img[i]):
#       timg=[[0 for i in range(w)] for j in range(h)]
#       for k in range(0,near+1):
#         for d in dr:
#           posy=i+d[0]*k
#           posx=j+d[1]*k
#           if posy>=0 and posy<h and posx>=0 and posx<w:
#             timg[posy][posx]=1
#             am[i*w+j][posy*w+posx]=1
#         print(timg)  

#   return am

# val=adjm(4,5,1)

In [179]:
with open('data/train/x_train.pkl','rb') as f:
        x1=pickle.load(f)
with open('data/train/y_train.pkl','rb') as f:
        y1=pickle.load( f)
with open('data/valid/x_valid.pkl','rb') as f:
        x2=pickle.load(f)
with open('data/valid/y_valid.pkl','rb') as f:
        y2=pickle.load( f)
# with open('data/test/x_test.pkl','rb') as f:
#         x3=pickle.load(f)
# with open('data/test/y_test.pkl','rb') as f:
#         y3=pickle.load( f)

In [180]:
with open('data/test/x_test.pkl','rb') as f:
        x3=pickle.load(f)
with open('data/test/y_test.pkl','rb') as f:
        y3=pickle.load( f)

In [181]:
x1.shape

(420, 9, 12)

In [182]:
print(max(y1))

6.0


In [183]:
class GCN(tf.keras.layers.Layer):
    def __init__(self,inp_shape=9*12,out_shape=9*12,h=9, w=12,near=3):
        super(GCN, self).__init__()
        self.am=[[0 for i in range(w*h)] for j in range(h*w)]
        self.img=[[0 for i in range(w)] for j in range(h)]
  
        self.dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

        for i,v in enumerate(self.img):
           for j,v2 in enumerate(self.img[i]):
        # timg=[[0 for i in range(w)] for j in range(h)]
              for k in range(0,near+1):
                for d in self.dr:
                  posy=i+d[0]*k
                  posx=j+d[1]*k
                  if posy>=0 and posy<h and posx>=0 and posx<w:
            # timg[posy][posx]=1
                    self.am[i*w+j][posy*w+posx]=1
        # print(timg) 
        self.am=np.array(self.am) 
        self.D = np.diag(np.sum(self.am, axis=0)) 
        self.Dinv=np.linalg.inv(self.D)
        self.am=tf.Variable(self.am,trainable=True,dtype=tf.float32,name="ajm")
        self.Dinv=tf.Variable(self.Dinv,trainable=True,dtype=tf.float32,name="dm")
        # self.norm=tf.matmul(self.Dinv*self.am)
        self.w = self.add_weight(
            shape=(inp_shape, out_shape), initializer="random_normal", trainable=True,dtype=tf.float32,name='adjw'
        )
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        inputs=tf.cast(inputs,dtype=tf.float32)
        # inputs=tf.transpose()
        print(inputs.shape)
        # print("inpshape",tf.transpose(inputs,perm=[1,0]).shape)
        # print(self.Dinv.shape)
        # print(self.am.shape)
        # print(self.w.shape)
        first_half=tf.matmul(inputs,tf.matmul(self.Dinv,self.am))
    
        return tf.matmul(first_half, self.w) 

In [184]:
def build_model(near=3,lay=2,bs=bs,n_out=7):
    inputs = tf.keras.Input( shape=(9,12))
    mlflow.log_param("layers", lay)
    mlflow.log_param("nearest", near)
    mlflow.set_tag("layer", "changed")
    print(inputs.shape)
    # x=tf.reshape(inputs,[inputs.shape[1]*inputs.shape[2]],name="initial reshape")
    x=tf.keras.layers.Flatten()(inputs)
    for i in range(lay):
        x=GCN(near=near)(x)
        x=tf.keras.layers.Activation('relu',name="graph_layer"+str(i+1))(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Dense(units=32,activation="relu",name="dense1")(x)
    output=tf.keras.layers.Dense(n_out,activation="softmax",name="out_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output, name="graph_neuralnetwork")

    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)
    return model

In [185]:
model=build_model()

MlflowException: Run with id=afad619951784d10b02a660cc75ff333 not found

In [ ]:
model.summary()

Model: "graph_neuralnetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9, 12)]           0         
                                                                 
 flatten (Flatten)           (None, 108)               0         
                                                                 
 gcn (GCN)                   (None, 108)               34992     
                                                                 
 graph_layer1 (Activation)   (None, 108)               0         
                                                                 
 gcn_1 (GCN)                 (None, 108)               34992     
                                                                 
 graph_layer2 (Activation)   (None, 108)               0         
                                                                 
 batch_normalization (BatchN  (None, 108)      

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="mode/base_model",
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [ ]:
# with mlflow.start_run(nested=true):
history = model.fit(x1,y1, batch_size=bs, epochs=epochs, callbacks=[callback],validation_data=(x2,y2),verbose=2)


Layer GCN has arguments ['self', 'inp_shape', 'out_shape', 'h', 'w', 'near']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/150
(10, 108)
(10, 108)
(10, 108)
(10, 108)


2022-06-26 21:58:17.807976: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-26 21:58:17.983518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


(10, 108)
(10, 108)
42/42 - 1s - loss: 2.0358 - accuracy: 0.1762 - val_loss: 25.1917 - val_accuracy: 0.1429 - 872ms/epoch - 21ms/step
Epoch 2/150


2022-06-26 21:58:18.569194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


42/42 - 0s - loss: 1.9269 - accuracy: 0.1905 - val_loss: 2.8952 - val_accuracy: 0.2286 - 353ms/epoch - 8ms/step
Epoch 3/150
42/42 - 0s - loss: 1.8775 - accuracy: 0.2405 - val_loss: 2.5265 - val_accuracy: 0.1000 - 355ms/epoch - 8ms/step
Epoch 4/150
42/42 - 0s - loss: 1.8484 - accuracy: 0.2690 - val_loss: 1.8844 - val_accuracy: 0.1714 - 322ms/epoch - 8ms/step
Epoch 5/150
42/42 - 0s - loss: 1.8262 - accuracy: 0.2143 - val_loss: 3.4422 - val_accuracy: 0.1429 - 392ms/epoch - 9ms/step
Epoch 6/150
42/42 - 0s - loss: 1.7972 - accuracy: 0.2167 - val_loss: 5.4988 - val_accuracy: 0.1429 - 317ms/epoch - 8ms/step
Epoch 7/150
42/42 - 0s - loss: 1.7776 - accuracy: 0.2310 - val_loss: 1.9551 - val_accuracy: 0.1429 - 316ms/epoch - 8ms/step
Epoch 8/150
42/42 - 0s - loss: 1.8169 - accuracy: 0.2524 - val_loss: 2.1486 - val_accuracy: 0.1500 - 315ms/epoch - 7ms/step
Epoch 9/150
42/42 - 0s - loss: 1.7873 - accuracy: 0.2405 - val_loss: 2.6139 - val_accuracy: 0.1571 - 313ms/epoch - 7ms/step
Epoch 10/150
42/42 -

2022-06-26 21:58:25.092529: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/v9/13sg2_p569zdm3chj4w3dmr00000gn/T/tmpvk06le_6/model/data/model/assets


In [ ]:
model.save('/Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/base_model/gcn1')

(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
INFO:tensorflow:Assets written to: /Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/base_model/gcn1/assets


In [186]:
model1=tf.keras.models.load_model("model/base_model/gcn1")

In [187]:
model1.summary()

Model: "graph_neuralnetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9, 12)]           0         
                                                                 
 flatten (Flatten)           (None, 108)               0         
                                                                 
 gcn (GCN)                   (None, 108)               34992     
                                                                 
 graph_layer1 (Activation)   (None, 108)               0         
                                                                 
 gcn_1 (GCN)                 (None, 108)               34992     
                                                                 
 graph_layer2 (Activation)   (None, 108)               0         
                                                                 
 batch_normalization (BatchN  (None, 108)      